In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM,Dense
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [ ]:
multi_data = pd.read_csv("data/flotation-cell.csv",parse_dates=True,index_col=0)

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# Look at the dataset because something is wrong ?

In [ ]:
multi_data.reset_index(inplace=True)
del multi_data['Date and time']

In [ ]:
multi_data.dropna(inplace=True)

In [ ]:
reframed_multi_data = series_to_supervised(multi_data.values)

In [ ]:
y = reframed_multi_data['var5(t)']

In [ ]:
x = reframed_multi_data[['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var1(t)', 'var2(t)', 'var3(t)', 'var5(t-1)','var4(t)']]

In [ ]:
x_train = x[:2500]
x_test = x[2500:]

In [ ]:
y_train = y[:2500]
y_test = y[2500:]

In [ ]:
# Something fishy here
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
y_train = scaler.fit_transform(y_train.values.reshape(-1,1))
y_test = scaler.fit_transform(y_test.values.reshape(-1,1))

x_train = x_train.reshape(x_train.shape[0],1,x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0],1,x_test.shape[1])

In [ ]:
x_train.shape

In [ ]:
# does hyper parameter tuning help ?
multi_lstm_model = Sequential()
multi_lstm_model.add(LSTM(100,input_shape=(x_train.shape[1],x_train.shape[2])))
multi_lstm_model.add(Dense(1))
multi_lstm_model.compile(loss="mse",optimizer="adam")

In [ ]:
multi_lstm_model.fit(x_train,y_train,nb_epoch=25,batch_size=100)

In [ ]:
y_pred = multi_lstm_model.predict(x_test)

In [ ]:
def plot_predictions(preds,y_test):
    plt.figure(figsize=(10, 5.5))
    plt.plot(preds,linestyle='-',color='b')
    plt.plot(y_test,linestyle='-',color='r')
    plt.legend(['Predicted','Actual'], loc=2)
    plt.title('Actual vs Predicted')

In [ ]:
plot_predictions(y_pred[:100],y_test[:100])

In [ ]:
r2_score(y_test,y_pred)